# Agregagador de CSV's

Agrega os csv's de cada órgão em um só csv (remuneracao_servidores.csv), criando os atributos 'ORGAO', 'ANO' e 'MES'. 

### Imports

In [8]:
import pandas as pd
import os
import re

## Funções

### Lista Diretórios

In [9]:
def lista_dirs(path):
    dirs=[]
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for direc in d:
            print(direc)
            if '.txt' not in direc:
                dirs.append(os.path.join(r, direc))
    return dirs

### Lista os arquivos de um diretório

In [3]:
def lista_arqs(path):
    files = []
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for file in f:
            if '.csv' in file:
                f_out = os.path.join(r, file)
                files.append(f_out)
    return files

### Cria o dataframe a partir do diretório de um órgão

In [25]:
def cria_df_orgao(path):
    files = lista_arqs(path)
    df_orgao = pd.DataFrame()
    for f in files:
        if '.csv' in f:
            df_mes = pd.read_csv(f, sep=';', header=0, decimal=',', engine='python')
            
            file_name = f.split("_")[1]
            orgao_name = path.split("/")[1]
            
            mes = file_name[4:6]
            ano = file_name[0:4]
            
            df_mes["ANO"] = ano
            df_mes["MES"] = mes
            
            df_orgao= pd.concat([df_orgao, df_mes], ignore_index=True)
            df_orgao["ORGAO"] = orgao_name
    try:
        df_orgao.drop(['Unnamed: 10'], axis=1, inplace=True)
    except:
        print('Sem Coluna Unnamed')
    return df_orgao

### Cria o dataframe geral

In [23]:
def cria_df_geral(path):
    df_geral = pd.DataFrame()
    dirs = lista_dirs(path)
    for d in dirs:
        if (str(d) == "csv/PM-ATIVOS" or str(d) == "csv/FUNDAÇÃO VILA OLIMPICA" or str(d) == "csv/PRODAM"):
            continue
        print("Processando diretório:", str(d))
        nome_dir = d.split("/")[1]
        ds_orgao = pd.DataFrame()
        df_orgao = cria_df_orgao(d)
        df_geral = pd.concat([df_geral,df_orgao], ignore_index=True)
        df_geral = trata_nan(df_geral)
    return df_geral

### Trata os valores NaN

In [11]:
def trata_nan(df):
    for col in df.columns:
        if(df[col].dtype == 'object'):
            df[col] = df[col].fillna('-')
        if(df[col].dtype == 'float64'):
            df[col] = df[col].fillna(0.00)
    
    return df

## Main

In [26]:
def main():
    path = 'csv/'
    df_geral = cria_df_geral(path)
    df_geral.to_csv(r'ds/remuneracao_servidores.csv')
    print (df_geral)
        
main()

CGE
PM-ATIVOS
SRMM
SEAD-PENSÃO HANSENIANOS
SEINFRA
SERIRA
CBMAM
SNPH
FMT-AM
SPF
SEIND
FAPEAM
SEPED
CETAM
ALFREDO DA MATA
FUNDAÇÃO AMAZONPREV
SEPROR
SEJUSC
SEAD
SSP
IPEM-AM
IDAM
FUNTEC
SEPLANCTI
IPAAM
UEA
PENSIONISTAS
SETRAB
SEDUC
SEJEL
IMPRENSA OFICIAL
PRODAM
UGPE
CGL
CASA CIVIL DO GOVERNO
SEC
FVS
SEAS
SEMA
FHEMOAM
DETRAN
FEI
ERGSP
SEAD-PENSÃO ESPECIAL II
FUNDAÇÃO VILA OLIMPICA
FEH
SECOM
CASA MILITAR
ADS
VICE-GOVERNADORIA
SUHAB
SERGB
JUCEA
FCECON
CB-CIVIS
POLICIA CIVIL
SUSAM
SEFAZ
AMAZONPREV
SEAD-PENSÃO ESPECIAL I
PGE
SEAP
ADAF
ARSAM
PM-CIVIS
FHAJ
OUVIDORIA GERAL
Processando diretório: csv/CGE


/home/vini/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Processando diretório: csv/SRMM
Processando diretório: csv/SEAD-PENSÃO HANSENIANOS
Processando diretório: csv/SEINFRA
Processando diretório: csv/SERIRA
Processando diretório: csv/CBMAM
Processando diretório: csv/SNPH
Processando diretório: csv/FMT-AM
Processando diretório: csv/SPF
Processando diretório: csv/SEIND
Sem Coluna Unnamed
Processando diretório: csv/FAPEAM
Processando diretório: csv/SEPED
Processando diretório: csv/CETAM
Processando diretório: csv/ALFREDO DA MATA
Processando diretório: csv/FUNDAÇÃO AMAZONPREV
Processando diretório: csv/SEPROR
Processando diretório: csv/SEJUSC
Processando diretório: csv/SEAD
Processando diretório: csv/SSP
Processando diretório: csv/IPEM-AM
Processando diretório: csv/IDAM
Processando diretório: csv/FUNTEC
Processando diretório: csv/SEPLANCTI
Processando diretório: csv/IPAAM
Processando diretório: csv/UEA
Processando diretório: csv/PENSIONISTAS
Processando diretório: csv/SETRAB
Processando diretório: csv/SEDUC
Processando diretório: csv/SEJEL
Pro